In [ ]:
# %pip install requests pandas google-cloud-bigquery prefect

In [33]:
from google.oauth2 import service_account
import seaborn as sns
import pandas as pd
import requests
import pandas_gbq as gbq
import logging
import os
from google.cloud import bigquery

## Extract Data Crypto Market 

In [3]:
def fetch_coingecko_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 20,      # Top 20 coin
        'page': 1,
        'sparkline': False
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    logging.info(f"Data fetched from {url}")
    df = pd.DataFrame(data)
    # Simpan ke CSV (optional)
    df.to_csv("../data/raw_crypto.csv", index=False)
    return df

## Transform Data

In [4]:
# Data enrich

def enrich_crypto_data(df):
    # 1. Selisih ke ATH
    df['ath_diff'] = df['ath'] - df['current_price']
    # 2. Gap persen ke ATH
    df['ath_gap_pct'] = ((df['ath'] - df['current_price']) / df['ath']) * 100
    # 3. Rasio volume ke market cap
    df['vol_to_market_cap'] = df['total_volume'] / df['market_cap']
    # 4. Kategori market cap
    df['cap_category'] = pd.cut(
        df['market_cap'],
        bins=[0, 1e9, 10e9, 1e13],
        labels=['Small Cap', 'Mid Cap', 'Large Cap']
    )
    # 5. Arah perubahan harga 24 jam terakhir
    df['change_direction'] = df['price_change_percentage_24h'].apply(
        lambda x: 'up' if x > 0 else ('down' if x < 0 else 'no change')
    )
    # 6. Status mendekati ATH atau tidak
    df['ath_status'] = df['ath_gap_pct'].apply(
        lambda x: 'near ATH' if x < 10 else ('far from ATH' if x > 30 else 'moderate')
    )
    return df

In [38]:
# Data Cleaning

def clean_and_enrich_crypto_data(input_csv="raw_crypto.csv", output_csv="clean_crypto.csv"):
    df = pd.read_csv(input_csv)
    keep_cols = [
        'id', 'symbol', 'name', 'current_price', 'market_cap', 'total_volume',
        'price_change_percentage_24h', 'last_updated', 'ath'
    ]
    df = df[keep_cols]
    df.dropna(inplace=True)
    df['last_updated'] = pd.to_datetime(df['last_updated'])
    df = enrich_crypto_data(df)
    df.to_csv(output_csv, index=False)
    return df


## Load to BigQuery

In [ ]:
# BigQuery configuration
PROJECT_ID = "api-crypto-dashboard"
KEY_PATH = "../config/api-crypto-dashboard-4d7f7e155f6b.json"
TABLE_ID = "API_CRYPTO_DASHBOARD.crypto_boys"

In [36]:
# set up credentials
SCOPES = ["https://www.googleapis.com/auth/bigquery"]
credentials = service_account.Credentials.from_service_account_file(KEY_PATH, scopes=SCOPES)

In [39]:
df = clean_and_enrich_crypto_data()

gbq.to_gbq(df, 
           destination_table=TABLE_ID, 
           project_id=PROJECT_ID, 
           credentials=credentials)

In [30]:
def load_to_bigquery(csv_path="../data/clean_crypto.csv"):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PortProject_API/config/api-crypto-dashboard-4d7f7e155f6b.json"
    df = pd.read_csv(csv_path)
    client = gbq.Client()
    table_id = "API_CRYPTO_DASHBOARD.crypto_markets"  

    job = client.load_table_from_dataframe(df, table_id, if_exists="replace")
    job.result()
    print(f"Sukses upload ke {table_id}")